In [2]:
first_run = False

In [3]:
import pandas as pd
import json
import glob
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np
from langdetect import detect
import matplotlib.dates as md
import plotly.express as px

%load_ext autoreload
%autoreload 2


### Structure
Content and narratives
- Topic modeling of EU Commission tweets [DONE]
- Which topics generate most engagement? Which topics generate most positive or negative engagement? Which narratives resonate the most? [DONE]

Styles
- How is the style of the European commission tweeting? Is it related to engagement? [TODO]

Emotional content
- What is the prevalent emotional content? [TODO]

Comparison
- Which are the best predictors of engagement?
    - Bag of words models [DONE]
    - Transformer-based models [TODO]
    - Stylistic predictors [TODO]
    - Emotion predictors [TODO]
    - Topic models [DONE]
   
### Meta:
- Extend to other languages?
- Extend to other accounts?

### Content and narratives

In [22]:
topic_20d = pd.read_json(f'logs/topic/{top_npmi[n_model][0]}/topic_preds_{int(top_npmi[n_model][1])}.jsonl',
                         orient='records', lines=True)
response_df = pd.read_json('processed/pre_topic_responses_sentiment.jsonl', 
                           orient='records', 
                           lines=True)   

In [23]:
# Display topic evolution
# Overview, engagement
# Overview sentiment of comments

### Styles

In [ ]:
import spacy
import textdescriptives as td
nlp = spacy.load("en_core_web_md")
nlp.add_pipe("textdescriptives") 
dfs = [] 
for t in topic_20d['text']:
    try: dfs.append(td.extract_df(nlp(t))) 
    except: print(t); pass

In [49]:
# Plot style distribution per topic
# Plot style over time
# Plot engagement by style

### Emotions

In [53]:
# Maybe do the same

### Predictive models

In [ ]:
# Comparative performance
# Alternatively, try regression